In [ ]:
import requests

def convert_epub_to_pdf(epub_path, pdf_path, api_key):
    url = 'https://api.cloudconvert.com/v2/convert'
    headers = {'Authorization': f'Bearer {api_key}'}
    payload = {
        'inputformat': 'epub',
        'outputformat': 'pdf',
        'input': 'upload',
        'file': open(epub_path, 'rb')
    }
    response = requests.post(url, headers=headers, files=payload)

    if response.status_code == 200:
        with open(pdf_path, 'wb') as f:
            f.write(response.content)
        print('Conversion successful')
    else:
        print('Conversion failed')

# Usage example
epub_file = 'input.epub'
pdf_file = 'output.pdf'
api_key = 'YOUR_API_KEY'

convert_epub_to_pdf(epub_file, pdf_file, api_key)
